In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
def sample_replay_prob(m, τ_l_m):
    # Schedule replays for all the block
    Replays = np.zeros((m-1, 8**m - 1 ))
    for s in range(τ_l_m + 1, τ_l_m + 8 ** m):
        for d in range(m):
            if s - τ_l_m % (8 ** d) == 0:
                p_s_d = np.sqrt(8 ** d / (s - τ_l_m))
                R_s_d = int(np.random.random() < p_s_d)
                Replays[s, d] = R_s_d
    return Replays

def MDBE(T):
    # Initialization
    l = 1
    t = 1
    m = 1
    
    while t <= T:
        # (★) Block handling
        if t == 1 or t == τ_l_m + 8 ** m: # if ending of the block without significant shift
            m += 1
            B_MASTER = set(range(1, 1 + 8**m)) # Reset MASTER set
            D_t = {m}
            τ_l_m = t

            # Initialize StoreActive dictionary
            StoreActive = {}
            StoreActive[m] = [τ_l_m, τ_l_m + 8**m]
            for d in range(m) :
                StoreActive[d] = []

            B_t = {}
            Replays = sample_replay_prob(m, τ_l_m)

        # Check if there are replays that are beginning in this precise round t
        for d in range(m) :
            if Replays[t, d] == 1 :
                D_t.add(d)
                StoreActive[d] = [t, t + 8**d]
                B_t[d] = set(range(1, 1 + 8**d))
        
        # Check if a replay ends
        for d in D_t:
            if StoreActive[d][1] == t :
                D_t.discard(d)
                B_t[d] = set()

        # Hierarchical sampling
        d_min = min(D_t)
        B_parent = random.choice(B_t[d_min])

        # faudrait creer une classe d'arbre...
        




        # Replay deactivation
        for d in range(m):
            if StoreActive.get((l, d), {}).get(t, None):
                D[l].discard(d)
                B_t[(l, d)] = set()

        # Bin selection
        d0 = min(D[l])
        B_parent = random.choice(list(B_t[(l, d0)]))
        xt = B_parent  # If no children

        children = get_children(B_parent, d0, B_t)
        if children:
            xt = random.choice(children)
            B_parent = xt

        # Bin eviction
        for d in D[l]:
            for B in B_t[(l, d)]:
                for d_ in [d_ for d_ in range(d + 1, m)]:
                    for B_prime in B_t[(l, d_)]:
                        if bin_condition(B_prime, B, StoreActive, l, d):
                            B_t[(l, d_)] -= {B_prime}

        # MASTER set update
        B_MASTER[(l, m)] = B_MASTER[(l, m)].intersection(B_t.get((l, m), set()))
        if not B_MASTER[(l, m)]:
            τ[(l + 1, 0)] = t + 1
            l += 1
            t = τ[(l, 0)]
            m = 0
            continue

        t += 1
